# Compare Facebook with survey data

In [1]:
import pandas as pd
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

## Load data

In [2]:
ad_estimates = pd.read_csv('../../data/query_results/hispanic_expats.tsv', sep='\t', index_col=False)
survey_data = pd.read_csv('../../data/pop_stats/CURRENT_Statistical-Portrait-of-the-Foreign-Born-2015_FINAL.txt', sep='\t', index_col=False)

In [4]:
ad_estimates.head(2)

,name,interests,ages_ranges,genders,behavior,scholarities,languages,family_statuses,relationship_statuses,geo_locations,household_composition,all_fields,targeting,response,dau_audience,mau_audience,timestamp,publisher_platforms
0,Hispanic Expats,NaN,{u'min': 18},0.0,"{u'or': [6003133212372L], u'name': u'Hispanic ...",NaN,NaN,NaN,NaN,"{u'country': u'US', u'values': [{u'key': u'384...",NaN,"(('genders', 0), ('ages_ranges', {u'min': 18})...",{'flexible_spec': [{'behaviors': [{'id': 60031...,"{""data"":[{""bid_estimate"":{""min_bid"":175,""media...",128621,180000,1526920227,"[""facebook""]"
1,Hispanic Expats,NaN,{u'min': 18},0.0,"{u'not': [6003133212372L], u'name': u'Non-Hisp...",NaN,NaN,NaN,NaN,"{u'country': u'US', u'values': [{u'key': u'384...",NaN,"(('genders', 0), ('ages_ranges', {u'min': 18})...","{'flexible_spec': [], 'publisher_platforms': [...","{""data"":[{""bid_estimate"":{""min_bid"":165,""media...",2024399,3000000,1526920227,"[""facebook""]"


In [5]:
survey_data.head(2)

,Unnamed: 0,Foreign born,Mexico,South and East Asia,Europe/ Canada,Caribbean,Central America,South America,Middle East,Sub-Saharan Africa,All other,Hispanic
0,California,"10,676,748","4,270,016","3,594,688","912,241","71,248","870,699","245,886","500,468","136,055","75,447","5,211,963"
1,Texas,"4,627,888","2,548,247","878,525","242,971","83,340","403,734","140,770","113,616","202,908","13,777","3,035,321"
